# Open Data Hub + Starburst Enterprise Platform:  <br>  Access and Explore your Data

## Introduction
**Open Data Hub**, a platform for data scientists and developers of intelligent applications, supports the full Machine Learning lifecycle by providing a robust, scalable platform and a flexible, interactive environment for teams to do their work. 

**Starburst Enterprise Platform** (formerly PrestoSQL) provides a secure and performant single point of access to all of your data without having to first copy or move it to a central repository. Starburst Enterprise Platform focuses on the first, and often most difficult problem teams face when starting a new project -- the acquisition and preparation of data. With Starburst Enterprise Platform, data scientists and developers will be able to quickly and easily combine data from multiple sources to perform comprehensive analyses for their organizations.

This demonstration illustrates how quickly a data scientist can access data and pull it into the Open Data Hub Jupyter environment using Starburst Enterprise Platform.

## Installing Required Packages
Open Data Hub provides images loaded with popular open source data science packages. These are also the notebook images we use when building our own intelligent applications. 

While these images typically have everything we need, we can always layer in specific package requirements using `pip` and a `requirements.txt` file. 

## Environment Initialization

### Import packages and environment variables
Let's import the package we installed in the previous step and assign environment variables we included while spawning our notebook. This way, we won't accidentally expose sensitive connection information!

Lastly, we use these variables and the `trino.dbapi.connect` function to create our Connection object. 

In [ ]:
import os

import pandas
import trino


TRINO_USERNAME = os.environ.get('TRINO_USERNAME', 'trino')
TRINO_PASSWORD = os.environ.get('TRINO_PASSWORD', 'trino')
TRINO_HOSTNAME = os.environ.get('TRINO_HOSTNAME', 'CHANGE_ME')
TRINO_PORT = os.environ.get('TRINO_PORT', '80')

In [ ]:
conn = trino.dbapi.connect(
    host=TRINO_HOSTNAME,
    port=TRINO_PORT,
    user=TRINO_USERNAME,
    http_scheme='http',
)

In [ ]:
def get_sql(sql, connector):
    """Return pandas DataFrame."""
    
    cur = connector.cursor()
    cur.execute(sql)
    response = pandas.DataFrame(
        cur.fetchall(), columns=[c[0] for c in cur.description]
    )
    return response

## What is Starburst Enterprise Platform and how does it work? 
Starburst Enterprise Platform is an incredibly efficient layer sitting between the data consumer and our data sources. It brings our data together so we can query sources individually or join them together in ways that previously required extensive ETL processes.

Let's use our connection object and SQL statements to interact with Starburst Enterprise Platform. 

The following SQL statements help us understand our data sources:
* `'SHOW CATALOGS'` shows the data sources available to us at this time,
* `'SHOW SCHEMAS'` indicates how data tables are organized, and 
* `'SHOW TABLES'` exposes datasets within a catalog and schema.

In [ ]:
sql = 'SHOW CATALOGS'
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = 'SHOW SCHEMAS from s3'
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = 'SHOW TABLES FROM s3.tpch'
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = 'DESCRIBE s3.tpch.customer'
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = 'SELECT * FROM s3.tpch.customer limit 40'
df = get_sql(sql, conn)
print(df)

**Please note**:
Your data sources will show up under `'SHOW CATALOGS'` after their respective connectors are configured in your catalog.

**Some useful terminology**:
* **data source** - your data stored in a database, bucket, or other. Starburst Enterprise Platform has connectors for most sources already. 
* **connector** - connectors configure your catalog. They give you access to your data sources. They are similar to drivers, in a way.
* **catalog** - defines schemas and properties of connections so Starburst Enterprise Platform knows how to query your data.
* **schema**  - how your tables are organized.
* **table**   - similar to tables in a relational database. A set of rows and columns representing your data based on connector properties.

## Accessing Data
We've seen the data sources we have access to (CATALOGS), their associated SCHEMAS, and some data tables from tpch.  Let's run some actual queries to access the data!

For the following example, we'll assume there are CSV files in an S3 bucket "test".

In [ ]:
sql = "CREATE SCHEMA s3.test WITH (location = 's3a://test/')"
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = """

CREATE TABLE s3.test.customers ( 
   customer_id BIGINT, 
   label TINYINT
)

"""
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = """

INSERT INTO s3.test.customers VALUES (43, 1)

"""
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = 'SHOW TABLES FROM s3.test'
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = 'DESCRIBE s3.test.customers'
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = 'SELECT * FROM s3.test.customers limit 40'
df = get_sql(sql, conn)
print(df)

In [ ]:
sql = 'DROP TABLE s3.test.customers'
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = 'DROP SCHEMA s3.test'
df = get_sql(sql, conn)
df.head()